In [ ]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore #for outlier
from scipy import stats # for plot normal distribution

In [ ]:
# Read the data
df= pd.read_csv('/kaggle/input/nycproperty/Merged.csv')

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns = [c.replace('\n', '') for c in df.columns]

df.head(10)

### Data Cleaning and Data Preprocessing 

In [ ]:
## Duplicates

In [ ]:
#Delete the duplicates and check that it worked
df = df.drop_duplicates(df.columns, keep='last')
sum(df.duplicated(df.columns))

In [ ]:
## Uniqueness

In [ ]:
#Check for the number of unique values for every column 
for i in df.columns:
  print(str(i) + ":" + str(df[i].nunique()))

In [ ]:
## Data type transformation 

Since there is no information in EASE-MENT columns, we will drop it initially. We will also drop the SALE DATE column as we are not considering the effect of time on the sale price. 

In [ ]:
#Converting columns to respective datatype
categoricalData = ['BOROUGH','TAX CLASS AT PRESENT','BUILDING CLASS CATEGORY',
                    'TAX CLASS AT TIME OF SALE',
                    'BUILDING CLASS AT PRESENT',
                    'BUILDING CLASSAT TIME OF SALE']
for each in categoricalData:
    df[each] = df[each].astype('category')
df['LAND SQUARE FEET'] = pd.to_numeric(df['LAND SQUARE FEET'], errors='coerce')
df['GROSS SQUARE FEET']= pd.to_numeric(df['GROSS SQUARE FEET'], errors='coerce')
df['SALE PRICE'] = df['SALE PRICE'].replace('[CA\$,]', '', regex=True).astype(float) * 0.77 # convert to US dollars 
df = df.drop(columns=["SALE DATE", "EASE-MENT"])
df.info()

In [ ]:
## Missing values

In [ ]:
# Drop LAND SQUARE FEET = 0, GROSS SQUARE FEET = 0, YEAR BUILT = 0   
df = df[df['LAND SQUARE FEET'] > 0]
df = df[df['GROSS SQUARE FEET'] > 0]
df = df[df['YEAR BUILT'] > 0]  
#Drop the missing values
df.dropna(how='all',axis=1,inplace=True)
df.info()

In [ ]:
##Check whether dropping nan worked 
df.isnull().sum()

In [ ]:
## Outliers

In this case, we will use Z score to detect the outliers in the columns of LAND SQUARE FEET, GROSS SQUARE FEET and SALE PRICE. 
Z score is a significant measure that tells how much a number is above or below the mean of the dataset in terms of standard deviation. We set the threshold=3

In [ ]:
df['landSquare_zscore']=zscore(df['LAND SQUARE FEET'])
df.loc[df['landSquare_zscore'].abs()<=3]
print("The count of outliers in LAND SQUARE FEET is: {}".format(len((np.where(zscore(df['LAND SQUARE FEET']) > 3)[0]))))

df['grossSquare_zscore']=zscore(df['GROSS SQUARE FEET'])
df.loc[df['grossSquare_zscore'].abs()<=3]
print("The count of outliers in GROSS SQUARE FEET is: {}".format(len((np.where(zscore(df['GROSS SQUARE FEET']) > 3)[0]))))

df['salePrice_zscore']=zscore(df['SALE PRICE'])
df.loc[df['salePrice_zscore'].abs()<=3]
print("The count of outliers in SALE PRICE is: {}".format(len((np.where(zscore(df['SALE PRICE']) > 3)[0]))))

### Exploratory Data Analysis

### Feature Engineering

In [ ]:
df['AGE'] = 2021 - df['YEAR BUILT']

# is the building pre or post war?
df['AGE_CATEGORY'] = np.where(df['AGE']>=76,1,0)

df['AGE_CATEGORY'] = df['AGE_CATEGORY'].astype('category')

df.head()

In [ ]:
df.drop('AGE',axis=1,inplace=True)

## Converting Borough column to name of each property where it is located.

In [ ]:
df['BOROUGH'].unique()

In [ ]:
df['BOROUGH'] = df['BOROUGH'].map({1.0:"Manhattan",
                                 2.0:"Bronx",
                                 3.0:"Brooklyn",
                                 4.0:"Queens",
                                 5.0:"Staten Island"})
df['BOROUGH'].unique()

### Target Variable[Sale Price]

##### Visualization of Raw Data

Calculating mean, median and mode for each column in dataframe

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
N = 'Nominal'
O = 'Ordinal'
I = 'Interval'
R = "Ratio"

In [ ]:
NOIR_Classification={"BOROUGH":N,
                    "NEIGHBORHOOD":N,
                    "BUILDING CLASS CATEGORY":O,
                    "TAX CLASS AT PRESENT":O,
                    "BLOCK":N,
                    "LOT":N,
                    "BUILDING CLASS AT PRESENT":N,
                    "ADDRESS":N,
                    "APARTMENT NUMBER":N,
                    "ZIP CODE":N,
                    "RESIDENTIALUNITS":N,
                    "COMMERCIALUNITS":N,
                    "TOTAL UNITS":N,
                    "LAND SQUARE FEET":N,
                    "GROSS SQUARE FEET":N,
                    "YEAR BUILT":O,
                    "TAX CLASS AT TIME OF SALE":O,
                    "BUILDING CLASSAT TIME OF SALE":N,
                    "SALE PRICE":N,
                    "SALE DATE":O,
                     'AGE': R,
                     'AGE_CATEGORY': O

                   }

In [ ]:
for key,value in NOIR_Classification.items():
    print(f"Column: {key}\t\t\t Classification Type :{value}")

In [ ]:
df['SALE PRICE'].describe()

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.boxplot(df['SALE PRICE'])
plt.title('Sale Price - Box Plot')
plt.subplot(1,2,2)
sns.distplot(df['SALE PRICE'])
plt.title('Sale Price - Density Plot')
# Skewness
print("Skewness: %f" % df['SALE PRICE'].skew())
plt.show()

##### Range set-up & Log-Transformation

In [ ]:
# Sale price percentage = $0
len(df.loc[df['SALE PRICE'] == 0 ])/len(df)

In [ ]:
# Sale price percentage < $50000
len(df.loc[df['SALE PRICE'] < 50000])/len(df)

In [ ]:
# Sale price percentage > 12000000
len(df.loc[df['SALE PRICE'] > 12000000 ])/len(df)

##### Observation: 

It is observed that a lot of sales occur with an absurdly small number: \\$0  most commonly ( 40%  of the sale price =  \\$0 ). 
On the basis of the original data resource, it is noted that these sales are in effect transfers of deeds between parties. For instance, the transfer of ownership of the house from parents to the child after the parents move out for retirement. To handle this situation, a reasonable range for the sale price will be set up. The instances for which the sale price is greater than \\$12M and less than \\$50000 will be removed since it will help eliminate the special cases. Following which, log transformation can be performed. 

In [ ]:
#Removing rows where year_built is equal to zero because these rows can be considered as outliers
df = df[df['YEAR BUILT']!=0]

In [ ]:
df = df[(df['SALE PRICE'] > 50000) & (df['SALE PRICE'] < 12000000)]
sns.distplot(df['SALE PRICE'])

In [ ]:
# log(x) transform

df["SALE PRICE"]=np.log(df["SALE PRICE"])

plt.subplots(figsize=(15,6))
plt.subplot(1,2,1)
sns.distplot(df["SALE PRICE"], fit = stats.norm)
plt.ylabel('Frequency')
(mu, sigma) = stats.norm.fit(df["SALE PRICE"])
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)], loc='best')

plt.subplot(1,2,2)
plt.boxplot(df['SALE PRICE'])
plt.title('Sale Price - Box Plot')

#### Feature Analysis and Selection 

In [ ]:
df = df.drop(['salePrice_zscore', 'landSquare_zscore', 'grossSquare_zscore'],axis =1)
df.columns

#### (i) Correlation 

In [ ]:
#Colinearity heatmap
corr = df.corr()
plt.figure(figsize=(12,6))
sns.heatmap(corr, annot=True)

#### (ii) Feature Analysis and Selection

a. BOROUGH

Explanation: The name of the borough in which the property is located.

1 = Bronx\
2 = Brooklyn\
3 = Manhattan\
4 = Queens\
5 = Staten Island

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Number of Observations in each BOROUGH")
df['BOROUGH'].value_counts().plot(kind='bar')

# Average price by borough
plt.subplot(1,2,2)
plt.title("Mean Sale Price in each BOROUGH")
df.groupby('BOROUGH').mean()['SALE PRICE'].plot(kind = 'bar')

In [ ]:
plt.figure(figsize=(15,6))
sns.boxplot(x=df['BOROUGH'].astype('category'), y = df['SALE PRICE'], data=pd.melt(df))
plt.show()

#### b. NEIGHBORHOOD

Explanation: While evaluating properties, the name of hte neighborhood is determined by the Department of Finance assessors. Although, the common name of the neighborhood is the same as designated by the Finance department, slight differences may occur when it comes to neighborhood boundary lines. It is also noted that a few sub-neighborhoods might not be included. 

In [ ]:
# Best 50 neighborhoods having largest average building sale price
plt.figure(figsize=(30,10))
df.groupby('NEIGHBORHOOD').mean()['SALE PRICE'].sort_values(ascending = False).head(50).plot(kind = 'bar')
plt.title("Best 50 neighborhoods with largest average building sale price")
plt.ylabel('Average sale price')

#### Building Class category

Explanation: To help the users of the Rolling Sales Files to identify similar properties by broad usage such as One Family Homes without having to look up Individual Buildign Classes, this filed is included. The files are sorted by Borough, Neighborhood, Block, Building Class Category and Lot.  

In [ ]:
df['BUILDING CLASS CATEGORY'].unique()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10,20), sharey=True)
# Average price by building class category
plt.subplot(211)
df.groupby('BUILDING CLASS CATEGORY').mean()['SALE PRICE'].plot(kind = 'bar')
plt.title('Average sale price of buildings in each building class categories in df')
plt.ylabel('Average sale price')

# number of buildings by building class category
plt.subplot(212)
df.groupby('BUILDING CLASS CATEGORY').size().plot(kind = 'bar')
plt.title('Number of buildings in each building class categories in df')
plt.ylabel('Number of buildings')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,6))
order = sorted(df['BUILDING CLASS CATEGORY'].unique())
sns.boxplot(x=df['BUILDING CLASS CATEGORY'], y = df['SALE PRICE'], data=df, order=order)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df.groupby('BUILDING CLASS CATEGORY').mean()['SALE PRICE'])
plt.title('Average sale price of different building classes in df')
plt.ylabel('Average sale price')
plt.xticks(rotation = 70)

plt.plot(df.groupby('BUILDING CLASSAT TIME OF SALE').mean()['SALE PRICE'],color='r')
plt.title('Average sale price of different building classes in df')
plt.ylabel('Average sale price')
plt.xticks(rotation = 70)

In [ ]:
df['TAX CLASS AT TIME OF SALE'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df['TAX CLASS AT TIME OF SALE'], y = df['SALE PRICE'], data=pd.melt(df))
plt.show()

In [ ]:
## Block and Lot

In [ ]:
plt.figure(figsize=(30,10))
order = df['BUILDING CLASSAT TIME OF SALE'].unique()
sns.boxplot(x='BUILDING CLASSAT TIME OF SALE', y='SALE PRICE', data=df, order = order)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.hist(df['YEAR BUILT'])
plt.title('Number of buildings built in different year in df')
plt.ylabel('Count')
plt.xlabel("Year Built")

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df.groupby('YEAR BUILT').mean()['SALE PRICE'])
plt.plot(df.groupby('YEAR BUILT').median()['SALE PRICE'],color='r')
plt.title('Average and Median Sale price of buildings built in different year in df')
plt.ylabel('Average or Median sale price')

In [ ]:
# Year Built
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)

sns.regplot(x='YEAR BUILT', y='SALE PRICE',data=df)
plt.title('Year Built vs. sale price ')
plt.subplot(1,2,2)
sns.boxplot(x= 'YEAR BUILT', data=df)
plt.title('Year Built boxplt')
plt.show()

In [ ]:
df['RESIDENTIALUNITS'].max()

In [ ]:
# residential_units
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
sns.regplot(x='RESIDENTIALUNITS', y='SALE PRICE',data=df)
plt.title('residential unit vs. sale price relationship')
plt.subplot(1,2,2)
sns.boxplot(x='RESIDENTIALUNITS', data=df)
plt.title('residential unit boxplt')
plt.show()

In [ ]:
#density plot
sns.distplot(df['RESIDENTIALUNITS'],fit = stats.norm)
plt.ylabel('Frequency')

In [ ]:
df['LAND SQUARE FEET']=np.log(df['LAND SQUARE FEET'])
df['GROSS SQUARE FEET']=np.log(df['GROSS SQUARE FEET'])
plt.figure(figsize=(15,6));
a=sns.jointplot(x="LAND SQUARE FEET", y="SALE PRICE", data=df,
                  kind="reg", truncate=False,
                  color="m", height=7);
b=sns.jointplot(x="GROSS SQUARE FEET", y="SALE PRICE", data=df,
                  kind="reg", truncate=False,
                  color="m", height=7);

### 3.3 Summary

Based on previous data visulization and analysis, we will drop these features: 

NEIGHBORHOOD, ADDRESS, APARTMENT NUMBER, BLOCK, LOT, ZIP CODE, BLOCK, TAX CLASS AS OF FINAL ROLL 18/19, 
BUILDING AS OF FINAL ROLL 18/19 

In [ ]:
df = df.drop(columns=["ADDRESS", "APARTMENT NUMBER", "LOT", "ZIP CODE", "BLOCK", 
                        "NEIGHBORHOOD", "TAX CLASS AT TIME OF SALE",
                        "BUILDING CLASSAT TIME OF SALE", "BUILDING CLASS AT PRESENT","YEAR BUILT"])

In [ ]:
df.info()

In [ ]:
#Colinearity heatmap
corr = df.corr()
plt.figure(figsize=(12,6))
sns.heatmap(corr, annot=True)

In [ ]:
categoricals= ['BOROUGH','TAX CLASS AT PRESENT',
               'BUILDING CLASS CATEGORY','AGE_CATEGORY']
one_hot_encoded = pd.get_dummies(df[categoricals])
df_new = df.drop(categoricals,axis = 1)
df_new = df_new.join(one_hot_encoded)
df_new.info()

In [ ]:
df_new

In [ ]:
#density plot
sns.distplot(df_new['LAND SQUARE FEET'],fit = stats.norm)
plt.ylabel('Frequency')
plt.title("log(LAND SQUARE FEET) density plot")

In [ ]:
#density plot
sns.distplot(df_new['GROSS SQUARE FEET'],fit = stats.norm)
plt.ylabel('Frequency')
plt.title("log(GROSS SQUARE FEET) density plot")

In [ ]:
from sklearn.model_selection import train_test_split

X = df_new.drop('SALE PRICE', axis = 1)
y = df_new['SALE PRICE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

# Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

In [ ]:
pred_linear = lr_model.predict(X_test)
plt.scatter(y_test, pred_linear)
plt.title('Residual plot of Linear Regression')
print('MSE:', metrics.mean_squared_error(y_test, pred_linear))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred_linear)))
print('R-square', metrics.r2_score(y_test, pred_linear))

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dr = DecisionTreeRegressor()
dr.fit(X_train,y_train)

In [ ]:
pred_dr = dr.predict(X_test)
plt.scatter(y_test, pred_dr)
plt.title('Residual plot of Decision Tree')
print('MSE:', metrics.mean_squared_error(y_test, pred_dr))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred_dr)))
print('R-square', metrics.r2_score(y_test, pred_dr))

# Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rg = RandomForestRegressor()
rg.fit(X_train,y_train)

In [ ]:
pred_rg = rg.predict(X_test)
plt.scatter(y_test, pred_rg)
plt.title('Residual plot of Random Forest')
print('MSE:', metrics.mean_squared_error(y_test, pred_rg))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred_rg)))
print('R-square', metrics.r2_score(y_test, pred_rg))

# XGBoost

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(tree_method="gpu_hist",enable_categorical=True)
xgb.fit(X_train,y_train)

In [ ]:
pred_xgb = xgb.predict(X_test)
plt.scatter(y_test, pred_xgb)
plt.title('Residual plot of XGBoost')
print('MSE:', metrics.mean_squared_error(y_test, pred_xgb))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred_xgb)))
print('R-square', metrics.r2_score(y_test, pred_xgb))

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
plot_importance(xgb)
pyplot.show()

In [ ]:
feature_important = xgb.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending= False)
data.nlargest(40, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features

# Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha = 0.1)
lasso_reg.fit(X_train, y_train)
lasso_reg_predict = lasso_reg.predict(X_test)

plt.scatter(y_test, lasso_reg_predict)
plt.title('Residual plot of LASSO')
print('MAE:', metrics.mean_absolute_error(y_test, lasso_reg_predict))
print('MSE:', metrics.mean_squared_error(y_test, lasso_reg_predict))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, lasso_reg_predict)))
print('R-square', metrics.r2_score(y_test, lasso_reg_predict))

# Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha = 0.1)
ridge_reg.fit(X_train, y_train)
ridge_reg_predict = ridge_reg.predict(X_test)

plt.scatter(y_test, ridge_reg_predict)
plt.title('Residual plot of Ridge Regression')
print('MAE:', metrics.mean_absolute_error(y_test, ridge_reg_predict))
print('MSE:', metrics.mean_squared_error(y_test, ridge_reg_predict))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, ridge_reg_predict)))
print('R-square', metrics.r2_score(y_test, ridge_reg_predict))

# Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=0.1, l1_ratio=0.9, selection='random', random_state=42)
model.fit(X_train, y_train)
elastic_predict = model.predict(X_test)

plt.scatter(y_test, elastic_predict)
plt.title('Residual plot of Elastic Net')
print('MAE:', metrics.mean_absolute_error(y_test, elastic_predict))
print('MSE:', metrics.mean_squared_error(y_test, elastic_predict))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, elastic_predict)))
print('R-square', metrics.r2_score(y_test, elastic_predict))


# Robust Regression

In [ ]:
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
from sklearn.linear_model import RANSACRegressor

model = RANSACRegressor(max_trials=1000)
model.fit(X_train, y_train)

test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

plt.scatter(y_test, test_pred)
plt.title('Residual plot of Robust Regression')

# 6 Feature Importance of the Best Model: XGBoost

In [ ]:

xgb_result = pd.DataFrame({'feature': X_train.columns, 'importance': abs(xgb.coef_)})
xgb_result_sorted = xgb_result.sort_values(by = 'importance', ascending = True)

fig, ax1 = plt.subplots(figsize = (16,25))
ax1.barh(xgb_result_sorted['feature'], xgb_result_sorted['importance'])
ax1.set_ylabel('feature importance')
ax1.set_title('Feature importance of Ridge')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,25))
ax1.barh(xgb_result_sorted['feature'], xgb_result_sorted['importance'])
ax1.set_ylabel('feature importance')
ax1.set_title('Feature importance of Ridge')
plt.show()

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as pyplot
# plot feature importance
plot_importance(xgb)
pyplot.show()

In [ ]:
X.info

In [ ]:
feature_important = xgb.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.nlargest(40, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features